##         Deep Learning Course Project - Gesture Recognition

Developing a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote. 

The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command:

Thumbs up:  Increase the volume
Thumbs down: Decrease the volume
Left swipe: 'Jump' backwards 10 seconds
Right swipe: 'Jump' forward 10 seconds  
Stop: Pause the movie

Each video is a sequence of 30 frames (or images).

And here we are using 2 architecture - 3D CNN and a CNN-RNN based model.

In [1]:
# importing all the libraries which might be required in the model
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

In [2]:
# importing all the libraries which might be required in the model
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In [3]:
# loading all the training and validation datasets
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())


In [4]:
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # lenght of the video frames
nb_channel = 3 # numbe rof channels in images 3 for color(RGB) and 1 for Gray

In [5]:
# Helper function to generate a random affine transform on the image
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [6]:
#data resizing - Resizing all the images, so we can have all the images in a specific size
def crop_resize_img(img):
    if img.shape[0] != img.shape[1]:
        img=img[0:120,10:150]
    resized_image = imresize(img, (nb_rows,nb_cols))
    return resized_image

In [7]:
# Helper function to normalise the data
def normalize_data(data):
    return data/127.5-1

In [8]:
# Helper function to initialize all the batch image data and labels
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [9]:
# A function to do the data Augmentation, and prepare the data for further processes in modeling
def data_augmentation(source_path, folder_list, batch_num, batch_size, t,validation):
    
    batch_data,batch_labels = init_batch_data(batch_size)
    
    # We will also build an augumented batch data with affine transformation
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)
    
    # We will also build an augmented batch data with horizontal flip
    batch_data_flip,batch_labels_flip = init_batch_data(batch_size)
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, nb_frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generate a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cropping non symmetric frames
            if image.shape[0] != image.shape[1]:
                image=image[0:120,20:140]
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            #Normal data
            batch_data[folder,idx] = (resized)
            
            #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))
            
            # Data with horizontal flip
            batch_data_flip[folder,idx]= np.flip(resized,1)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
        if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
        elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                    
        else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
                  
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)

    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 
    batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

In [10]:
## In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well 
## as create a batch of video frames. 

def generator(source_path, folder_list, batch_size, validation=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield data_augmentation(source_path, folder_list, batch, batch_size, t,validation)
            

        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield data_augmentation(source_path, folder_list, batch, batch_size, t,validation)

In [11]:
# storing the train and validation paths and storing the train and validation sequesnces length
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)
num_classes = 5
print('# Frames =', nb_frames)


# training sequences = 663
# validation sequences = 100
# epochs = 20
# Frames = 30


## Using 3D CNN Architecture

In [139]:
# importing all the libraries required for 3D CNN model and also building an 3D CNN model here

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))


In [140]:
# using the right optimiser and also compling the model

optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 16, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_13 (Batc (None, 16, 120, 120, 8)   32        
_________________________________________________________________
activation_17 (Activation)   (None, 16, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 8, 60, 60, 8)      0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 8, 60, 60, 16)     3472      
_________________________________________________________________
batch_normalization_14 (Batc (None, 8, 60, 60, 16)     64        
_________________________________________________________________
activation_18 (Activation)   (None, 8, 60, 60, 16)     0         
__________

### Model - 1 -- Framesize = 30, batch_size = 10, epoch = 20

In [ ]:
# model creation 

batch_size = 10

In [27]:
# creating the train and validation generator
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [31]:
model_name_1 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_1):
    os.mkdir(model_name_1)
        
filepath = model_name_1 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [32]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [33]:
# giving right number of Epoch and fiting the model

num_epochs = 20
print("Total Params:", model.count_params())

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Total Params: 864101
Source path =  ./Project_data/val ; batch size =Source path =  ./Project_data/train  10
; batch size = 10
Epoch 1/20
67/67 [==============================] - 192s 3s/step - loss: 1.6936 - categorical_accuracy: 0.2783 - val_loss: 1.4282 - val_categorical_accuracy: 0.3900

Epoch 00001: saving model to model_init_2020-05-2413_34_13.508854/model-00001-1.69240-0.28004-1.42818-0.39000.h5
Epoch 2/20
67/67 [==============================] - 26s 386ms/step - loss: 1.4484 - categorical_accuracy: 0.3765 - val_loss: 1.3240 - val_categorical_accuracy: 0.5300

Epoch 00002: saving model to model_init_2020-05-2413_34_13.508854/model-00002-1.44836-0.37645-1.32402-0.53000.h5
Epoch 3/20
67/67 [==============================] - 29s 429ms/step - loss: 1.4268 - categorical_accuracy: 0.3947 - val_loss: 1.3466 - val_categorical_accuracy: 0.4900

Epoch 00003: saving model to model_init_2020-05-2413_34_13.508854/model-00003-1.42676-0.39469-1.34661-0.49000.h5
Epoch 4/20
67/67 [==============

### Model - 2 -- Framesize = 30, batch_size = 30, epoch = 30

In [34]:
# storing a new batch value
batch_size = 30

In [35]:
# creating train and validation generator
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [36]:
model_name_2 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_2):
    os.mkdir(model_name_2)
        
filepath = model_name_2 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [37]:
# using new epoch size and fiting the model
num_epochs = 30
print("Total Params:", model.count_params())

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Total Params: 864101
Source path =  ./Project_data/val ; batch size = 30
Source path =  ./Project_data/train ; batch size = 30
Epoch 1/30
23/23 [==============================] - 90s 4s/step - loss: 0.3367 - categorical_accuracy: 0.8721 - val_loss: 0.3769 - val_categorical_accuracy: 0.8500

Epoch 00001: saving model to model_init_2020-05-2413_34_13.508854/model-00001-0.33448-0.87582-0.37691-0.85000.h5
Epoch 2/30
23/23 [==============================] - 9s 370ms/step - loss: 0.4239 - categorical_accuracy: 0.8986 - val_loss: 0.4758 - val_categorical_accuracy: 0.8250

Epoch 00002: saving model to model_init_2020-05-2413_34_13.508854/model-00002-0.42393-0.89855-0.47576-0.82500.h5
Epoch 3/30
23/23 [==============================] - 9s 382ms/step - loss: 0.3357 - categorical_accuracy: 0.8647 - val_loss: 0.4443 - val_categorical_accuracy: 0.8000

Epoch 00003: saving model to model_init_2020-05-2413_34_13.508854/model-00003-0.33569-0.86473-0.44430-0.80000.h5

Epoch 00003: ReduceLROnPlateau red


Epoch 00026: saving model to model_init_2020-05-2413_34_13.508854/model-00026-0.37947-0.84058-0.34749-0.87500.h5
Epoch 27/30
23/23 [==============================] - 10s 415ms/step - loss: 0.2933 - categorical_accuracy: 0.8889 - val_loss: 0.5382 - val_categorical_accuracy: 0.8250

Epoch 00027: saving model to model_init_2020-05-2413_34_13.508854/model-00027-0.29334-0.88889-0.53816-0.82500.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 3.051757957450718e-08.
Epoch 28/30
23/23 [==============================] - 9s 407ms/step - loss: 0.2597 - categorical_accuracy: 0.9227 - val_loss: 0.3837 - val_categorical_accuracy: 0.8750

Epoch 00028: saving model to model_init_2020-05-2413_34_13.508854/model-00028-0.25968-0.92271-0.38369-0.87500.h5
Epoch 29/30
23/23 [==============================] - 10s 444ms/step - loss: 0.3250 - categorical_accuracy: 0.8792 - val_loss: 0.3619 - val_categorical_accuracy: 0.8250

Epoch 00029: saving model to model_init_2020-05-2413_34_13.508854/model-0

### Model-3 -- Framesize = 30, batch_size = 20, epoch = 20

In [38]:
# using new batch size
batch_size = 20

In [39]:
# creating train and validation generators
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [40]:
# creating checkpoint
model_name_3 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_3):
    os.mkdir(model_name_3)
        
filepath = model_name_3 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [41]:
# using new epoch value and fiting the model
num_epochs = 20

print("Total Params:", model.count_params())


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Total Params: 864101
Source path =  ./Project_data/valSource path =  ./Project_data/train Epoch 1/20
 ; batch size = ; batch size = 20
20
34/34 [==============================] - 90s 3s/step - loss: 0.2964 - categorical_accuracy: 0.8940 - val_loss: 0.4249 - val_categorical_accuracy: 0.8500

Epoch 00001: saving model to model_init_2020-05-2413_34_13.508854/model-00001-0.30362-0.89140-0.42494-0.85000.h5
Epoch 2/20
34/34 [==============================] - 13s 394ms/step - loss: 0.2968 - categorical_accuracy: 0.9020 - val_loss: 0.4254 - val_categorical_accuracy: 0.8500

Epoch 00002: saving model to model_init_2020-05-2413_34_13.508854/model-00002-0.29679-0.90196-0.42536-0.85000.h5
Epoch 3/20
34/34 [==============================] - 16s 476ms/step - loss: 0.3265 - categorical_accuracy: 0.8725 - val_loss: 0.4259 - val_categorical_accuracy: 0.8500

Epoch 00003: saving model to model_init_2020-05-2413_34_13.508854/model-00003-0.32653-0.87255-0.42586-0.85000.h5

Epoch 00003: ReduceLROnPlateau r

### Model 4 -- Framesize = 20, batch_size = 10, epoch = 20

In [117]:
# new batch size
batch_size = 10

In [118]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [119]:
# creating a checkpoint
model_name_4 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_4):
    os.mkdir(model_name_4)
        
filepath = model_name_4 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [120]:
print('# Frames =', nb_frames)

# Frames = 20


In [122]:
#using new epoch value and fiting the model
num_epochs = 20
nb_frames = 20

print("Total Params:", model.count_params())


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Total Params: 864101
Epoch 1/20
67/67 [==============================] - 61s 905ms/step - loss: 1.6875 - categorical_accuracy: 0.2775 - val_loss: 1.3339 - val_categorical_accuracy: 0.4400

Epoch 00001: saving model to model_init_2020-05-2414_12_25.543099/model-00001-1.69698-0.27221-1.33387-0.44000.h5
Epoch 2/20
67/67 [==============================] - 19s 276ms/step - loss: 1.4163 - categorical_accuracy: 0.4063 - val_loss: 1.5128 - val_categorical_accuracy: 0.4000

Epoch 00002: saving model to model_init_2020-05-2414_12_25.543099/model-00002-1.41628-0.40630-1.51276-0.40000.h5
Epoch 3/20
67/67 [==============================] - 18s 274ms/step - loss: 1.3418 - categorical_accuracy: 0.4776 - val_loss: 1.1230 - val_categorical_accuracy: 0.5500

Epoch 00003: saving model to model_init_2020-05-2414_12_25.543099/model-00003-1.34181-0.47761-1.12298-0.55000.h5
Epoch 4/20
67/67 [==============================] - 18s 269ms/step - loss: 1.3953 - categorical_accuracy: 0.4063 - val_loss: 1.3288 - va

### Model - 5 -- Framesize = 30, batch_size = 20, epoch = 20

In [12]:
# importing some libraries and creating a new model by adding some new layers to the model
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))


In [13]:
# using the adam optimiser
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

In [22]:
# using new batch size for a new architecture
batch_size = 30

In [23]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [24]:
# creating the checkpoint
model_name_5 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_5):
    os.mkdir(model_name_5)
        
filepath = model_name_5 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [25]:
# new epochs and fiting the model
num_epochs = 40

print("Total Params:", model.count_params())


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Total Params: 887605
Source path =  ./Project_data/val ; batch size = 30
Source path =  ./Project_data/train ; batch size = 30
Epoch 1/40
23/23 [==============================] - 155s 7s/step - loss: 1.5915 - categorical_accuracy: 0.2135 - val_loss: 1.5203 - val_categorical_accuracy: 0.1900

Epoch 00001: saving model to model_init_2020-05-2506_37_23.930997/model-00001-1.59304-0.21317-1.52031-0.19000.h5
Epoch 2/40
23/23 [==============================] - 11s 464ms/step - loss: 1.6011 - categorical_accuracy: 0.2899 - val_loss: 1.5904 - val_categorical_accuracy: 0.2750

Epoch 00002: saving model to model_init_2020-05-2506_37_23.930997/model-00002-1.60111-0.28986-1.59042-0.27500.h5
Epoch 3/40
23/23 [==============================] - 11s 467ms/step - loss: 1.6218 - categorical_accuracy: 0.2512 - val_loss: 1.4853 - val_categorical_accuracy: 0.4500

Epoch 00003: saving model to model_init_2020-05-2506_37_23.930997/model-00003-1.62179-0.25121-1.48530-0.45000.h5
Epoch 4/40
23/23 [==============

### Model - 6 -- Framesize = 20, batch_size = 20, epoch = 30

In [127]:
# using new batch size
batch_size = 20

In [128]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [129]:
# creating a checkpoint
model_name_6 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_6):
    os.mkdir(model_name_6)
        
filepath = model_name_6 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [130]:
# using new number of epochs and fiting the model
num_epochs = 30

print("Total Params:", model.count_params())


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Total Params: 864101
Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/30
34/34 [==============================] - 61s 2s/step - loss: 0.4294 - categorical_accuracy: 0.8362 - val_loss: 0.4819 - val_categorical_accuracy: 0.8000

Epoch 00001: saving model to model_init_2020-05-2414_12_25.543099/model-00001-0.42747-0.83761-0.48195-0.80000.h5
Epoch 2/30
34/34 [==============================] - 10s 283ms/step - loss: 0.4935 - categorical_accuracy: 0.7974 - val_loss: 0.4786 - val_categorical_accuracy: 0.8000

Epoch 00002: saving model to model_init_2020-05-2414_12_25.543099/model-00002-0.49352-0.79739-0.47860-0.80000.h5
Epoch 3/30
34/34 [==============================] - 11s 312ms/step - loss: 0.3994 - categorical_accuracy: 0.8595 - val_loss: 0.4849 - val_categorical_accuracy: 0.8000

Epoch 00003: saving model to model_init_2020-05-2414_12_25.543099/model-00003-0.39943-0.85948-0.48487-0.80000.h5
Epoch 4/30
34/34 [===============

34/34 [==============================] - 11s 319ms/step - loss: 0.4954 - categorical_accuracy: 0.8039 - val_loss: 0.4765 - val_categorical_accuracy: 0.8000

Epoch 00027: saving model to model_init_2020-05-2414_12_25.543099/model-00027-0.49536-0.80392-0.47649-0.80000.h5
Epoch 28/30
34/34 [==============================] - 10s 283ms/step - loss: 0.4020 - categorical_accuracy: 0.8562 - val_loss: 0.4821 - val_categorical_accuracy: 0.8000

Epoch 00028: saving model to model_init_2020-05-2414_12_25.543099/model-00028-0.40201-0.85621-0.48210-0.80000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 1.1920929521291868e-10.
Epoch 29/30
34/34 [==============================] - 11s 310ms/step - loss: 0.4271 - categorical_accuracy: 0.8203 - val_loss: 0.4809 - val_categorical_accuracy: 0.8000

Epoch 00029: saving model to model_init_2020-05-2414_12_25.543099/model-00029-0.42708-0.82026-0.48091-0.80000.h5
Epoch 30/30
34/34 [==============================] - 11s 321ms/step - loss: 0.5051 -

### Model - 7 -- Framesize = 16, batch_size = 20, epoch = 20

In [141]:
# using new batch size
batch_size = 20

In [142]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [143]:
# creating a new check point
model_name_7 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_7):
    os.mkdir(model_name_7)
        
filepath = model_name_7 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [144]:
# using new epoch number and fiting the model
num_epochs = 20

print("Total Params:", model.count_params())


if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Total Params: 864101
Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20
34/34 [==============================] - 51s 1s/step - loss: 1.8065 - categorical_accuracy: 0.2615 - val_loss: 1.4462 - val_categorical_accuracy: 0.3800

Epoch 00001: saving model to model_init_2020-05-2414_33_36.758578/model-00001-1.81465-0.25691-1.44623-0.38000.h5
Epoch 2/20
34/34 [==============================] - 6s 186ms/step - loss: 1.5156 - categorical_accuracy: 0.3301 - val_loss: 1.5360 - val_categorical_accuracy: 0.3400

Epoch 00002: saving model to model_init_2020-05-2414_33_36.758578/model-00002-1.51557-0.33007-1.53600-0.34000.h5
Epoch 3/20
34/34 [==============================] - 6s 186ms/step - loss: 1.5991 - categorical_accuracy: 0.3105 - val_loss: 1.4909 - val_categorical_accuracy: 0.3300

Epoch 00003: saving model to model_init_2020-05-2414_33_36.758578/model-00003-1.59907-0.31046-1.49086-0.33000.h5

Epoch 00003: ReduceLROnPlateau red

## Using CNN-RNN Architecture

In [30]:
# importing the libraries for CNN-RNN architecture
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout,LSTM
from keras.layers.convolutional import Conv3D,Conv2D, MaxPooling3D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.layers.recurrent import LSTM, GRU

In [31]:
# creating the new model by using CNN and RNN layers
nb_featuremap = [8,16,32,64]
nb_dense = [128,64,5]
nb_classes = 5
# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

model = Sequential()

model.add(TimeDistributed(Conv2D(nb_featuremap[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))


model.add(TimeDistributed(Conv2D(nb_featuremap[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
model.add(GRU(128, return_sequences=False))
model.add(Dense(nb_classes, activation='softmax')) 

### Model - 8 - CNN-RNN - frame size 30, Batch size 10, epoch - 30

In [22]:
# using a new batch size and using Adam optimizer 
batch_size = 10
from keras.optimizers import Adam

optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

# creating the `train_generator` and the `val_generator` which will be used in `.fit_generator`
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 15, 15, 64)    8256      
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 7, 7, 64)      0         
__________

In [23]:
# creating a checkpoint
model_name_8 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_8):
    os.mkdir(model_name_8)
        
filepath = model_name_8 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [24]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [25]:
# fiting the model
print("Total Params:", model.count_params())
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs,
                    verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, 
                    class_weight=None,
                    workers=1, initial_epoch=0,use_multiprocessing=True)

Total Params: 499093
Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/30
67/67 [==============================] - 213s 3s/step - loss: 1.2542 - categorical_accuracy: 0.4531 - val_loss: 1.6564 - val_categorical_accuracy: 0.4067

Epoch 00001: saving model to model_init_2020-05-2414_47_43.025626/model-00001-1.26461-0.44746-1.65639-0.40667.h5
Epoch 2/30
67/67 [==============================] - 74s 1s/step - loss: 1.1781 - categorical_accuracy: 0.5174 - val_loss: 1.3580 - val_categorical_accuracy: 0.4700

Epoch 00002: saving model to model_init_2020-05-2414_47_43.025626/model-00002-1.17811-0.51741-1.35804-0.47000.h5
Epoch 3/30
67/67 [==============================] - 72s 1s/step - loss: 1.1828 - categorical_accuracy: 0.5257 - val_loss: 1.2839 - val_categorical_accuracy: 0.5167

Epoch 00003: saving model to model_init_2020-05-2414_47_43.025626/model-00003-1.18281-0.52570-1.28393-0.51667.h5
Epoch 4/30
67/67 [====================

67/67 [==============================] - 72s 1s/step - loss: 0.2362 - categorical_accuracy: 0.9320 - val_loss: 0.6890 - val_categorical_accuracy: 0.7633

Epoch 00030: saving model to model_init_2020-05-2414_47_43.025626/model-00030-0.23624-0.93201-0.68901-0.76333.h5


### Model - 9 - CNN-RNN - frame size 30, Batch size 20, epoch - 20

In [26]:
batch_size = 20
from keras.optimizers import Adam

optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

# creating the `train_generator` and the `val_generator` which will be used in `.fit_generator`
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 15, 15, 64)    8256      
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 7, 7, 64)      0         
__________

In [27]:
# creating the checkpoint and filepath
model_name_9 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_9):
    os.mkdir(model_name_9)
        
filepath = model_name_9 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [28]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [29]:
# printing the number of paramerets in the model
print("Total Params:", model.count_params())
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs,
                    verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, 
                    class_weight=None,
                    workers=1, initial_epoch=0,use_multiprocessing=True)

Total Params: 499093
Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20
34/34 [==============================] - 304s 9s/step - loss: 0.2081 - categorical_accuracy: 0.9313 - val_loss: 1.0802 - val_categorical_accuracy: 0.7067

Epoch 00001: saving model to model_init_2020-05-2414_47_43.025626/model-00001-0.21232-0.92961-1.08020-0.70667.h5
Epoch 2/20
34/34 [==============================] - 47s 1s/step - loss: 0.1691 - categorical_accuracy: 0.9379 - val_loss: 1.7116 - val_categorical_accuracy: 0.5967

Epoch 00002: saving model to model_init_2020-05-2414_47_43.025626/model-00002-0.16910-0.93791-1.71160-0.59667.h5
Epoch 3/20
34/34 [==============================] - 49s 1s/step - loss: 0.3862 - categorical_accuracy: 0.8595 - val_loss: 2.1339 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to model_init_2020-05-2414_47_43.025626/model-00003-0.38624-0.85948-2.13387-0.44000.h5
Epoch 4/20
34/34 [====================

### Model - 10 - CNN-RNN - frame size 20, Batch size 10, epoch - 30

In [ ]:
batch_size = 10
from keras.optimizers import Adam
# using Adam optimiser
optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

# creating the `train_generator` and the `val_generator` which will be used in `.fit_generator`
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
# creating a checkpoint and a file path
model_name_10 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_10):
    os.mkdir(model_name_10)
        
filepath = model_name_10 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [32]:
# printing the number of parameters
print("Total Params:", model.count_params())
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs,
                    verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, 
                    class_weight=None,
                    workers=1, initial_epoch=0,use_multiprocessing=True)

Total Params: 499093
Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/30
67/67 [==============================] - 141s 2s/step - loss: 1.3600 - categorical_accuracy: 0.3964 - val_loss: 1.4789 - val_categorical_accuracy: 0.4433

Epoch 00001: saving model to model_init_2020-05-2218_33_33.160458/model-00001-1.35805-0.39819-1.47892-0.44333.h5
Epoch 2/30
67/67 [==============================] - 48s 710ms/step - loss: 1.2267 - categorical_accuracy: 0.5025 - val_loss: 1.5807 - val_categorical_accuracy: 0.3767

Epoch 00002: saving model to model_init_2020-05-2218_33_33.160458/model-00002-1.22674-0.50249-1.58069-0.37667.h5
Epoch 3/30
67/67 [==============================] - 49s 726ms/step - loss: 1.4445 - categorical_accuracy: 0.4129 - val_loss: 1.4712 - val_categorical_accuracy: 0.4333

Epoch 00003: saving model to model_init_2020-05-2218_33_33.160458/model-00003-1.44455-0.41294-1.47116-0.43333.h5
Epoch 4/30
67/67 [==============

### Model - 11 - CNN-RNN - frame size 20, Batch size 20, epoch - 20

In [33]:
batch_size = 20
from keras.optimizers import Adam

optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

# creating the `train_generator` and the `val_generator` which will be used in `.fit_generator`
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 20, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 15, 15, 64)    8256      
_________________________________________________________________
time_distributed_7 (TimeDist (None, 20, 7, 7, 64)      0         
__________

In [34]:
# creating a check point, and a file path
model_name_11 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_11):
    os.mkdir(model_name_11)
        
filepath = model_name_11 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [35]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [36]:
# printing the number of parameters
print("Total Params:", model.count_params())
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs,
                    verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, 
                    class_weight=None,
                    workers=1, initial_epoch=0,use_multiprocessing=True)

Total Params: 499093
Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20
34/34 [==============================] - 147s 4s/step - loss: 0.3295 - categorical_accuracy: 0.8867 - val_loss: 1.4224 - val_categorical_accuracy: 0.5500

Epoch 00001: saving model to model_init_2020-05-2218_33_33.160458/model-00001-0.33446-0.88386-1.42238-0.55000.h5
Epoch 2/20
34/34 [==============================] - 29s 848ms/step - loss: 0.4508 - categorical_accuracy: 0.8529 - val_loss: 1.7612 - val_categorical_accuracy: 0.5467

Epoch 00002: saving model to model_init_2020-05-2218_33_33.160458/model-00002-0.45085-0.85294-1.76117-0.54667.h5
Epoch 3/20
34/34 [==============================] - 29s 860ms/step - loss: 0.6299 - categorical_accuracy: 0.7908 - val_loss: 2.7543 - val_categorical_accuracy: 0.3500

Epoch 00003: saving model to model_init_2020-05-2218_33_33.160458/model-00003-0.62985-0.79085-2.75427-0.35000.h5
Epoch 4/20
34/34 [==============

### Model - 12 - CNN-RNN - LSTM- frame size 30, Batch size 10, epoch - 30

In [32]:
# creating a new CNN-RNN model by adding some new extra layers to existing model
nb_featuremap = [8,16,32,64]
nb_dense = [128,64,5]
nb_classes = 5
# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

model = Sequential()

model.add(TimeDistributed(Conv2D(nb_featuremap[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))


model.add(TimeDistributed(Conv2D(nb_featuremap[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
## model.add(GRU(128, return_sequences=False))
## model.add(Dense(nb_classes, activation='softmax')) 

model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.25))
        
model.add(Dense(nb_dense[1],activation='relu'))
model.add(Dropout(0.25))
        
model.add(Dense(nb_classes, activation='softmax'))

In [31]:
# using Adam optimiser and a batch size of 10
batch_size = 10
from keras.optimizers import Adam

optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

# creating the `train_generator` and the `val_generator` which will be used in `.fit_generator`
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_10 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_11 (TimeDis (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_12 (TimeDis (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_14 (TimeDis (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 30, 15, 15, 64)    8256      
_________________________________________________________________
time_distributed_16 (TimeDis (None, 30, 7, 7, 64)      0         
__________

In [32]:
# creating a checkpoint
model_name_12 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_12):
    os.mkdir(model_name_12)
        
filepath = model_name_12 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [33]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [34]:
# print the total number of parameters and fitting the model
print("Total Params:", model.count_params())
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs,
                    verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, 
                    class_weight=None,
                    workers=1, initial_epoch=0,use_multiprocessing=True)

Total Params: 531733
Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/30
67/67 [==============================] - 212s 3s/step - loss: 1.3688 - categorical_accuracy: 0.3761 - val_loss: 1.2730 - val_categorical_accuracy: 0.4833

Epoch 00001: saving model to model_init_2020-05-2414_47_43.025626/model-00001-1.37506-0.37305-1.27301-0.48333.h5
Epoch 2/30
67/67 [==============================] - 73s 1s/step - loss: 1.3768 - categorical_accuracy: 0.4229 - val_loss: 1.3526 - val_categorical_accuracy: 0.4500

Epoch 00002: saving model to model_init_2020-05-2414_47_43.025626/model-00002-1.37678-0.42289-1.35259-0.45000.h5
Epoch 3/30
67/67 [==============================] - 73s 1s/step - loss: 1.3170 - categorical_accuracy: 0.4660 - val_loss: 1.3638 - val_categorical_accuracy: 0.4000

Epoch 00003: saving model to model_init_2020-05-2414_47_43.025626/model-00003-1.31701-0.46600-1.36382-0.40000.h5
Epoch 4/30
67/67 [====================

67/67 [==============================] - 74s 1s/step - loss: 0.8412 - categorical_accuracy: 0.7032 - val_loss: 0.9657 - val_categorical_accuracy: 0.6767

Epoch 00030: saving model to model_init_2020-05-2414_47_43.025626/model-00030-0.84122-0.70315-0.96567-0.67667.h5


### Model - 13 - CNN-RNN - LSTM- frame size 30, Batch size 20, epoch - 20

In [18]:
# using Adam Optizer and using batch size as 20
batch_size = 20
from keras.optimizers import Adam

optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

# creating the `train_generator` and the `val_generator` which will be used in `.fit_generator`
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 15, 15, 64)    8256      
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 7, 7, 64)      0         
__________

In [19]:
# creating the checkpoint
model_name_13 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_13):
    os.mkdir(model_name_13)
        
filepath = model_name_13 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [20]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [21]:
# printing the number of parameters and fiting the model
print("Total Params:", model.count_params())
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs,
                    verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, 
                    class_weight=None,
                    workers=1, initial_epoch=0,use_multiprocessing=True)

Total Params: 499093
Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/20
34/34 [==============================] - 295s 9s/step - loss: 1.0589 - categorical_accuracy: 0.5681 - val_loss: 1.0425 - val_categorical_accuracy: 0.5900

Epoch 00001: saving model to model_init_2020-05-2417_27_20.041494/model-00001-1.04501-0.57667-1.04247-0.59000.h5
Epoch 2/20
34/34 [==============================] - 49s 1s/step - loss: 0.9478 - categorical_accuracy: 0.6340 - val_loss: 1.8359 - val_categorical_accuracy: 0.4333

Epoch 00002: saving model to model_init_2020-05-2417_27_20.041494/model-00002-0.94781-0.63399-1.83591-0.43333.h5
Epoch 3/20
34/34 [==============================] - 50s 1s/step - loss: 1.0710 - categorical_accuracy: 0.6209 - val_loss: 1.8938 - val_categorical_accuracy: 0.4433

Epoch 00003: saving model to model_init_2020-05-2417_27_20.041494/model-00003-1.07099-0.62092-1.89380-0.44333.h5
Epoch 4/20
34/34 [====================

### Model - 14 - CNN-RNN - LSTM- frame size 30, Batch size 20, epoch - 50

In [33]:
batch_size = 20
from keras.optimizers import Adam

# using Adam optimizer
optimiser =Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

# creating the `train_generator` and the `val_generator` which will be used in `.fit_generator`
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_10 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_11 (TimeDis (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_12 (TimeDis (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_14 (TimeDis (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 30, 15, 15, 64)    8256      
_________________________________________________________________
time_distributed_16 (TimeDis (None, 30, 7, 7, 64)      0         
__________

In [34]:
# creating the checkpoint
model_name_14 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_14):
    os.mkdir(model_name_14)
        
filepath = model_name_14 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [35]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [36]:
# printing the number of parameters and fiting the model
print("Total Params:", model.count_params())
num_epochs = 50
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs,
                    verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, 
                    class_weight=None,
                    workers=1, initial_epoch=0,use_multiprocessing=True)

Total Params: 531733
Source path =  ./Project_data/val ; batch size = 20
Source path =  ./Project_data/train ; batch size = 20
Epoch 1/50
34/34 [==============================] - 268s 8s/step - loss: 1.3976 - categorical_accuracy: 0.3454 - val_loss: 1.8120 - val_categorical_accuracy: 0.3200

Epoch 00001: saving model to model_init_2020-05-2506_37_23.930997/model-00001-1.40517-0.34289-1.81203-0.32000.h5
Epoch 2/50
34/34 [==============================] - 50s 1s/step - loss: 1.3576 - categorical_accuracy: 0.4248 - val_loss: 2.0048 - val_categorical_accuracy: 0.3067

Epoch 00002: saving model to model_init_2020-05-2506_37_23.930997/model-00002-1.35759-0.42484-2.00481-0.30667.h5
Epoch 3/50
34/34 [==============================] - 46s 1s/step - loss: 1.5315 - categorical_accuracy: 0.3301 - val_loss: 1.6098 - val_categorical_accuracy: 0.2467

Epoch 00003: saving model to model_init_2020-05-2506_37_23.930997/model-00003-1.53146-0.33007-1.60976-0.24667.h5
Epoch 4/50
34/34 [====================

34/34 [==============================] - 48s 1s/step - loss: 0.9370 - categorical_accuracy: 0.6340 - val_loss: 1.0899 - val_categorical_accuracy: 0.5233

Epoch 00030: saving model to model_init_2020-05-2506_37_23.930997/model-00030-0.93704-0.63399-1.08995-0.52333.h5
Epoch 31/50
34/34 [==============================] - 46s 1s/step - loss: 0.8978 - categorical_accuracy: 0.6699 - val_loss: 1.0671 - val_categorical_accuracy: 0.5333

Epoch 00031: saving model to model_init_2020-05-2506_37_23.930997/model-00031-0.89780-0.66993-1.06711-0.53333.h5
Epoch 32/50
34/34 [==============================] - 46s 1s/step - loss: 0.9402 - categorical_accuracy: 0.6471 - val_loss: 1.0601 - val_categorical_accuracy: 0.5267

Epoch 00032: saving model to model_init_2020-05-2506_37_23.930997/model-00032-0.94017-0.64706-1.06010-0.52667.h5
Epoch 33/50
34/34 [==============================] - 45s 1s/step - loss: 0.9593 - categorical_accuracy: 0.6078 - val_loss: 1.0767 - val_categorical_accuracy: 0.5200

Epoch 000

### Model - 15 - CNN-RNN - LSTM- frame size 20, Batch size 15, epoch - 35

In [58]:
# creating a new model by adding new extra layers to the existing model
nb_featuremap = [8,16,32,64,128]
nb_dense = [128,64,5]
nb_classes = 5
# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

model = Sequential()

model.add(TimeDistributed(Conv2D(nb_featuremap[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))


model.add(TimeDistributed(Conv2D(nb_featuremap[1], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[2], (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(nb_featuremap[3], (2,2),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))

model.add(TimeDistributed(Flatten()))

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
## model.add(GRU(128, return_sequences=False))
## model.add(Dense(nb_classes, activation='softmax')) 

model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.25))
        
model.add(Dense(nb_dense[1],activation='relu'))
model.add(Dropout(0.25))
        
model.add(Dense(nb_classes, activation='softmax'))

In [59]:
# using a new batch value and using Adam optimizer
batch_size = 15
from keras.optimizers import Adam

optimiser =Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

# creating the `train_generator` and the `val_generator` which will be used in `.fit_generator`
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_30 (TimeDis (None, 30, 60, 60, 8)     224       
_________________________________________________________________
time_distributed_31 (TimeDis (None, 30, 60, 60, 16)    1168      
_________________________________________________________________
time_distributed_32 (TimeDis (None, 30, 30, 30, 16)    0         
_________________________________________________________________
time_distributed_33 (TimeDis (None, 30, 30, 30, 32)    4640      
_________________________________________________________________
time_distributed_34 (TimeDis (None, 30, 15, 15, 32)    0         
_________________________________________________________________
time_distributed_35 (TimeDis (None, 30, 15, 15, 64)    8256      
_________________________________________________________________
time_distributed_36 (TimeDis (None, 30, 7, 7, 64)      0         
__________

In [60]:
# creating a new checkpoint
model_name_15 = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name_15):
    os.mkdir(model_name_15)
        
filepath = model_name_15 + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:999: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [61]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [62]:
# printing the number of parameters and fiting the model
print("Total Params:", model.count_params())
num_epochs = 35
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs,
                    verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, 
                    class_weight=None,
                    workers=1, initial_epoch=0,use_multiprocessing=True)

Total Params: 224661
Source path =  ./Project_data/val ; batch size = 15
Source path =  ./Project_data/train ; batch size = 15
Epoch 1/35
45/45 [==============================] - 228s 5s/step - loss: 1.4272 - categorical_accuracy: 0.3270 - val_loss: 1.7484 - val_categorical_accuracy: 0.2167

Epoch 00001: saving model to model_init_2020-05-2418_36_25.990496/model-00001-1.41558-0.32881-1.74838-0.21667.h5
Epoch 2/35
45/45 [==============================] - 47s 1s/step - loss: 1.4289 - categorical_accuracy: 0.3481 - val_loss: 2.0322 - val_categorical_accuracy: 0.2429

Epoch 00002: saving model to model_init_2020-05-2418_36_25.990496/model-00002-1.42893-0.34815-2.03222-0.24286.h5
Epoch 3/35
45/45 [==============================] - 48s 1s/step - loss: 1.4298 - categorical_accuracy: 0.3852 - val_loss: 2.1547 - val_categorical_accuracy: 0.1857

Epoch 00003: saving model to model_init_2020-05-2418_36_25.990496/model-00003-1.42983-0.38519-2.15466-0.18571.h5
Epoch 4/35
45/45 [====================